## 08-Aug-2024 == Olympics 

#### Tim Spann @PaaSDev

### Milvus - Attu

![milvuslogo](https://milvus.io/images/milvus_logo.svg)


### CODE + COMMUNITY

Please join my meetup group NJ/NYC/Philly/Virtual. 

[https://www.meetup.com/unstructured-data-meetup-new-york/](https://www.meetup.com/unstructured-data-meetup-new-york/)


#### Contact Us

Get Milvused!  [https://milvus.io/](https://milvus.io/)

Read my Newsletter every week! [https://github.com/tspannhw/FLiPStackWeekly/blob/main/142-17June2024.md](https://github.com/tspannhw/FLiPStackWeekly/blob/main/142-17June2024.md)

For more cool Unstructured Data, AI and Vector Database videos check out the Milvus vector database videos here
[https://www.youtube.com/@MilvusVectorDatabase/videos](https://www.youtube.com/@MilvusVectorDatabase/videos)

#### Unstructured Data Meetups 

[https://www.meetup.com/pro/unstructureddata/](https://www.meetup.com/pro/unstructureddata/)
[https://zilliz.com/community/unstructured-data-meetup](https://zilliz.com/community/unstructured-data-meetup)
[https://zilliz.com/event](https://zilliz.com/event)

#### [https://x.com/milvusio](Twitter/X)  

#### [https://www.linkedin.com/company/zilliz/](LinkedIn)

#### [https://discord.com/invite/FjCMmaJng6](Discord)

#### [https://milvusio.medium.com/](Blog)

#### Please star our [https://github.com/milvus-io/milvus](Github)

#### [https://www.youtube.com/@FLaNK-Stack](Youtube)

#### [https://medium.com/@tspann/subscribe](Blog)

In [1]:
### wikipedia-api
### https://wikipedia-api.readthedocs.io/en/latest/README.html
### https://github.com/tspannhw/FLaNK-python-processors
### https://medium.com/cloudera-inc/building-a-library-of-python-processors-6b5517404a58

!pip3 install wikipedia-api

In [20]:
import wikipediaapi

WIKIPAGE = "2024_Summer_Olympics"

# wiki_wiki = wikipediaapi.Wikipedia(user_agent='NiFi tim.spann@zilliz',language='en',extract_format=wikipediaapi.ExtractFormat.WIKI)
wiki_wiki = wikipediaapi.Wikipedia(user_agent='NiFi tim.spann@zilliz',language='en',extract_format=wikipediaapi.ExtractFormat.HTML)

results = wiki_wiki.page(WIKIPAGE)
### print(str(results.text))
print(results)

wikidata = str(results.text)

print(len(results.text))
print(results.title)


2024_Summer_Olympics (id: ??, ns: 0)
29339
2024 Summer Olympics


In [23]:

text = str(results.text[:64000])
title = str(results.title)

print(len(text))
print(title)

29339
2024 Summer Olympics


In [24]:
import json
import requests
from pymilvus import model
from pymilvus.model.dense import SentenceTransformerEmbeddingFunction

model = SentenceTransformerEmbeddingFunction('all-MiniLM-L6-v2',device='cpu' )


In [40]:
import os
from pymilvus import MilvusClient

DIMENSION = 384 
MILVUS_URL = "https://in05-7bd87b945683c8d.serverless.gcp-us-west1.cloud.zilliz.com" 
COLLECTION_NAME = "olympics"
TOKEN = ""
# -----------------------------------------------------------------------------
# Connect to Milvus

# Local Docker Server
milvus_client = MilvusClient( uri=MILVUS_URL, token=TOKEN )

In [41]:
from pymilvus import connections
from pymilvus import utility
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
import pprint

## schema

schema = milvus_client.create_schema(
    auto_id=True,
    enable_dynamic_field=False
)

schema.add_field(field_name='id', datatype=DataType.INT64, is_primary=True, auto_id=True)
schema.add_field(field_name="title", datatype=DataType.VARCHAR,max_length=65535)
schema.add_field(field_name="text", datatype=DataType.VARCHAR,max_length=65535)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=DIMENSION)

index_params = milvus_client.prepare_index_params()

index_params.add_index(
    field_name="id",
    index_type="STL_SORT"
)

index_params.add_index(
    field_name="vector",
    index_type="IVF_FLAT",
    metric_type="L2",
    params={"nlist": 100}
)

milvus_client.create_collection(
    collection_name = COLLECTION_NAME,
    schema=schema,
    index_params=index_params,
)

res = milvus_client.get_load_state(
    collection_name = COLLECTION_NAME
)

print(res)

{'state': <LoadState: Loaded>}


In [42]:
data = []

data.append({"vector": model(text),"title": title,"text": text}, )
res = milvus_client.insert(collection_name=COLLECTION_NAME, data=data)
print(res)

{'insert_count': 1, 'ids': [451372946840723047], 'cost': 8}


In [44]:
# Define search parameters
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10}
}

# Use first record as search record
query_vector = [data[0]["vector"]]

# Execute the search on the 'vector' field
search_results = milvus_client.search(
    COLLECTION_NAME,
    data=query_vector,
    anns_field="vector",
    output_fields=["id", "title", "text"],
    search_params=search_params,
    limit=5
)

# Print search results
for hits in search_results:
    for hit in hits:
        print(f"Hit: {hit}")


Hit: {'id': 451372946840723047, 'distance': 0.0, 'entity': {'id': 451372946840723047, 'title': '2024 Summer Olympics', 'text': '<link rel="mw-deduplicated-inline-style" href="mw-data:TemplateStyles:r1236090951">\n<p class="mw-empty-elt">\n</p>\n\n<p class="mw-empty-elt">\n\n</p>\n\n<link rel="mw-deduplicated-inline-style" href="mw-data:TemplateStyles:r1129693374">\n<p>The <b>2024 Summer Olympics</b>, officially the <b>Games of the XXXIII Olympiad</b> and branded as <b>Paris 2024</b>, is an international multi-sport event taking place from 24\xa0July to 11\xa0August 2024 in France, with the opening ceremony having taken place on 26 July. Paris is the host city, with events held in 16 additional cities spread across Metropolitan France, and one subsite in Tahiti, French Polynesia.\n</p><p>Paris was awarded the Games at the 131st IOC Session in Lima, Peru, on 13\xa0September 2017. After multiple withdrawals that left only Paris and Los Angeles in contention, the International Olympic Comm